In [ ]:
import os
import easyocr
import pandas as pd
from tqdm import tqdm
import re
import torch

# --- CONFIGURATION ---
# The root directory containing all your experiment folders (exp1, exp2, etc.)
ROOT_DIR = "dataset_eval"
OUTPUT_CSV = "ocr_accuracy_results.csv"

def get_ground_truth(filename):
    """Extracts the expected onomatopoeia text from the filename."""
    # Normalize filename for consistent matching
    name_part = filename.split('_')[0].lower()
    
    if "byur" in name_part:
        return "byur"
    if "dutt" in name_part: # Matches Dutt, Duttt, etc.
        return "duuttt"
    if "gluduk" in name_part:
        return "gluduk gluduk gluduk"
    if "gong" in name_part: # Matches Gong, Gongg, etc.
        return "gonggg"
    if "ting" in name_part:
        return "ting"
    if "toktok" in name_part:
        return "tok tok tok"
    return None

def normalize_text(text):
    """Cleans and normalizes text for comparison."""
    # Convert to lowercase, remove all non-alphanumeric characters except spaces
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # Collapse multiple spaces into one
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def main():
    """Main function to run OCR on all images and evaluate accuracy."""
    if not os.path.exists(ROOT_DIR):
        print(f"Error: Root directory '{ROOT_DIR}' not found.")
        return

    print("Initializing EasyOCR... This might take a moment.")
    # Initialize the OCR reader for English
    reader = easyocr.Reader(['en'])
    
    all_results = []
    
    # Walk through all subdirectories in the root folder
    for dirpath, _, filenames in os.walk(ROOT_DIR):
        # Filter for image files
        image_files = [f for f in filenames if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not image_files:
            continue

        experiment_name = os.path.basename(dirpath)
        print(f"\nProcessing experiment: {experiment_name}...")
        
        for filename in tqdm(image_files, desc=f"Reading text from {experiment_name}"):
            image_path = os.path.join(dirpath, filename)
            ground_truth = get_ground_truth(filename)
            
            if ground_truth is None:
                continue

            try:
                # Run OCR on the image
                ocr_result = reader.readtext(image_path, detail=0)
                
                # Combine and normalize recognized text parts
                recognized_text = " ".join(ocr_result)
                normalized_recognized = normalize_text(recognized_text)
                normalized_truth = normalize_text(ground_truth)
                
                # Check for a match
                is_correct = 1 if normalized_recognized == normalized_truth else 0
                
                all_results.append({
                    "experiment": experiment_name,
                    "filename": filename,
                    "ground_truth": ground_truth,
                    "recognized_text": recognized_text,
                    "is_correct": is_correct
                })
            except Exception as e:
                print(f"Could not perform OCR on {filename}: {e}")
                all_results.append({
                    "experiment": experiment_name,
                    "filename": filename,
                    "ground_truth": ground_truth,
                    "recognized_text": f"ERROR: {e}",
                    "is_correct": 0
                })
            finally:
                # CRITICAL FIX: Clear GPU memory cache after each image
                # This helps prevent memory overload that can crash the kernel
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

    if not all_results:
        print("No images were processed.")
        return
        
    # Create and save a DataFrame
    df = pd.DataFrame(all_results)
    df.to_csv(OUTPUT_CSV, index=False)
    
    # Calculate and print overall accuracy
    overall_accuracy = df['is_correct'].mean() * 100
    
    print("\n--- OCR Evaluation Complete ---")
    print(f"Detailed results saved to {OUTPUT_CSV}")
    print(f"Overall OCR Accuracy: {overall_accuracy:.2f}%")
    
    # Print accuracy per experiment
    print("\nAccuracy per experiment:")
    accuracy_by_exp = df.groupby('experiment')['is_correct'].mean().reset_index()
    accuracy_by_exp['is_correct'] *= 100
    accuracy_by_exp.rename(columns={'is_correct': 'accuracy_%'}, inplace=True)
    print(accuracy_by_exp)

if __name__ == "__main__":
    main()



ModuleNotFoundError: No module named 'easyocr'